# irisで機械学習

アイリスデータセットを用いて、花の4種の特徴量から3種類のアヤメに分類する。

## 1. データの準備

まずは学習用データと評価用データを作る。

In [ ]:
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

# データセットを読み込んでDataFrameにする
iris = load_iris()
df = pd.DataFrame(iris.data, columns=iris.feature_names)

# データ分割
X_train, X_test, y_train, y_test = train_test_split(df, iris.target)

評価の実行と、評価結果（正答率）を表示する関数を用意しておく。

参考用に学習データでも評価を行う。

In [ ]:
def print_eval(model):
    # 学習データで評価し、正答率を表示
    print('train:', model.score(X_train, y_train))
    # 評価データで評価、正答率を表示
    print('test :', model.score(X_test, y_test))

## 2. 教師あり学習

scikit-learnのAPIを使用した機械学習は一般的に以下のような構成となる。

0. 学習データと評価データを用意する（「1. データの準備」で示した）
1. モデルを作る
2. 学習を行う
3. 結果を確認する

### 2-1. 線形回帰

線形回帰モデルで機械学習を行うプログラムを示す。

モデルは[LinearRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)を使用する。ここではデフォルトで使用しているため、オプションの指定はない、

In [ ]:
from sklearn.linear_model import LinearRegression

# モデルを作る
clf = LinearRegression()

# 学習を行う
clf.fit(X_train, y_train)

# 学習データと評価データで評価を行う
print_eval(clf)

その他、線形回帰固有の結果を出力する。

In [ ]:
print('回帰係数:', clf.coef_)
print('切片   :', clf.intercept_)
print('決定係数:', clf.score(X_train, y_train))

注1）再度機械学習を実行する場合は、JupyterLabのカーネルをリセットする必要がある。<br>
注2）機械学習のモデルは、学習を行うたびに異なる評価結果になることがある。

### 2-2. ロジスティック回帰

モデルは[LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)を使用する。オプションの`solve='lbfgs'`はL-BFGS法の使用を指定している。

In [ ]:
# ロジスティック回帰モデル：solver引数には最適化手法を指定
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(solver='lbfgs')
clf.fit(X_train, y_train)

print_eval(clf)

### 2-3. サポートベクターマシン

サポートベクターマシンは[SVC](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC)を使用する。ここではデフォルトで使用しているため、オプションの指定はない、

SVCの評価結果の取り出しは線形回帰等と異なるため、新たに`print_eval2()`を定義する。

In [ ]:
from sklearn import metrics

def print_eval2(model):
    # 学習データで評価し、結果を表示
    pre_train = clf.predict(X_train)
    print('train:', metrics.accuracy_score(y_train, pre_train))
    # 評価データで評価、結果を表示
    pre_test = clf.predict(X_test)
    print('test: ', metrics.accuracy_score(y_test, pre_test))

In [ ]:
from sklearn import svm

# モデル作成
clf = svm.SVC(kernel='linear')  # 線形で指定

# 学習の実行
clf.fit(X_train, y_train)

# 評価の実行と結果表示
print_eval2(clf)

### 2-4. 決定木（デシジョンツリー）

モデルは[DecisionTreeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html)を使用する。

In [ ]:
from sklearn import tree

# モデル作成（最大深さ3）
clf = tree.DecisionTreeClassifier(max_depth=3) # 決定木モデル（最大深さ3）

# 学習の実行
clf = clf.fit(X_train, y_train)

# 評価の実行と結果表示
print_eval2(clf)

学習時に生成された決定木を表示する。

In [ ]:
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree

plt.figure(figsize=(15, 10))
plot_tree(clf, feature_names=iris.feature_names, filled=True)
plt.show()

### 2-5. ランダムフォレスト

モデルは[RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)を使用する。

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# モデル作成
clf = RandomForestClassifier(random_state=0, n_estimators=10)

# 学習の実行
clf = clf.fit(X_train, y_train)

# 評価の実行と結果表示
print_eval2(clf)

### 2-6. ニューラルネットワーク

準伝播型ニューラルネットワークの一種である、多層パーセプトロン(Multilayer perceptron, MLP)を使用したプログラムを示す。

モデルは[MLPClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html)を使用する。

In [ ]:
from sklearn.neural_network import MLPClassifier

# モデル作成
clf = MLPClassifier(max_iter=1000)  # 最大繰り返し回数を1000回

# 学習の実行
clf.fit(X_train, y_train)

# 評価の実行と結果表示
print_eval2(clf)

損失カーブをグラフ化する。

損失カーブとは、教師データと予測データの差を、繰り返し回数ごとに示したもの。

In [ ]:
import matplotlib.pyplot as plt
import japanize_matplotlib

plt.title('損失カーブ')
plt.plot(clf.loss_curve_)
plt.xlabel("繰り返し回数")
plt.ylabel("損失")
plt.grid()
plt.show()

### 2-7. ナイーブベイズ（単純ベイズ）

ナイーブベイズは３種類あり、それぞれを以下に示す。

#### 2-7-1. ガウスモデル (Gaussian naive Bayes)

モデルは[GaussianNB](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html)を使用する。

In [ ]:
from sklearn.naive_bayes import GaussianNB

# モデル作成
clf = GaussianNB()

# 学習の実行
clf.fit(X_train, y_train)

# 評価の実行と結果表示
print_eval2(clf)

#### 2-7-2. 多項分布モデル (Multinomial naive Bayes)

モデルは[MultinomialNB](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html)を使用する。

In [ ]:
from sklearn.naive_bayes import MultinomialNB

# モデル作成
clf = MultinomialNB()

# 学習の実行
clf.fit(X_train, y_train)

# 評価の実行と結果表示
print_eval2(clf)

#### 2-7-3. ベルヌーイ分布モデル (Bernoulli naive Bayes)

モデルは[BernoulliNB](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.BernoulliNB.html)を使用する。

In [ ]:
from sklearn.naive_bayes import BernoulliNB

# モデル作成
clf = BernoulliNB()

# 学習の実行
clf.fit(X_train, y_train)

# 評価の実行と結果表示
print_eval2(clf)

### 2-8. k近傍法

モデルは[KNeighborsClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html)を使用する。

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# モデル作成
clf = KNeighborsClassifier()

# 学習の実行
clf.fit(X_train, y_train)

# 評価の実行と結果表示
print_eval2(clf)

## 3. 教師なし学習

### 3-1. k平均法

クラスタリングを行う手法。

モデルは[KMeans](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html)を使用する。

In [ ]:
from sklearn.cluster import KMeans

# モデルを作成
clf = KMeans(n_clusters=3)  # ３つのクラスタに分類する

# 学習
clf.fit(X_train)

# クラスタを予測
y_km = clf.predict(X_test)
y_km

クラスタの予測と評価データを接続し、seabornで分布を見て確認する。

In [ ]:
import seaborn as sns

def plot_result(y_km):
    df = X_test.copy()
    df['cluster'] = y_km
    sns.pairplot(df, hue='cluster', palette={0:'red', 1:'green', 2:'blue'})

plot_result(y_km)

ちなみに正解は以下の通り。

In [ ]:
df_tmp = X_test.copy()
df_tmp['cluster'] = y_test
sns.pairplot(df_tmp, hue='cluster', palette={0:'red', 1:'green', 2:'blue'})
plt.show()

### 3-2. 主成分分析 (PCA)

変換行列を生成して、次元削減を行う。

モデルは[PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html)を使用する。

対象データの傾向を見ることが目的なので、データを学習と評価に分けず、全体を使用する。

In [ ]:
import numpy as np
from sklearn.decomposition import PCA

# モデル作成
pca = PCA(n_components=2)
pca.fit(df)

# 変換行列を確認
pca.components_.T

In [ ]:
# 元のデータを２次元に変換
X = pca.transform(df)

# 散布図で表示
plt.scatter(x=X[:, 0], y=X[:, 1], c=iris.target)
plt.show()

### 3-3. t-SNE

次元削減を行う。

モデルは[TSNE](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html)を使用する。

In [ ]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2, random_state=42)
iris_tsne = tsne.fit_transform(iris.data)

# グラフ化
plt.figure(figsize=(10, 10))
sns.scatterplot(x=iris_tsne[:, 0],
                y=iris_tsne[:, 1],
                hue=iris.target, 
                palette=sns.color_palette("hsv", 3))
plt.show()